# maude-magic

> Fill in a module description here

**Refs:**
- [pexpect](https://pexpect.readthedocs.io/en/stable/)
- [nbdev](https://nbdev.fast.ai/tutorials/tutorial.html)
- [IPython magics](https://ipython.readthedocs.io/en/stable/config/custommagics.html)
- See persistent Shells.ipynb  

In [2]:
%%bash 
for package in pexpect nbdev jupyterlab-quarto
do  
    pip list | grep   "^$package *" > /dev/null && echo "$package found" || pip install $package
done
# This must be done in terminal
# nbdev_install_quarto

pexpect found
nbdev found
jupyterlab-quarto found


## The Maude Interpreter class

In [1]:
#| default_exp maude-magic

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
import os
import pexpect

In [228]:
#| export
timeout = 3

class TimeoutException(Exception):
    """Exception raised if time-out."""

    def __init__(self):
        super().__init__("Timeout exception.")
        self.error_code = 1

    def __str__(self):
        return f"{self.message} (Error Code: {self.error_code})"    


class MaudeInterpreter:
    """Controls maude execution, executing commands and print responses.
       Preserve sessions between different cell executions."""
    def __init__(self,debug=False,timeout=timeout):
        """ Init variables and spawns maude. """
        self.debug,self.timeout = debug,timeout
        self._control_request='pwd'
        self._control_response='pwd.*Maude> '
        #print(self._prompt)
        # Execute maude in environment "env". Add current working directory to MAUDE_LIB
        env = dict(os.environ)
        if not 'MAUDE_LIB' in env: raise(Exception('MAUDE_LIB environmet variable not found'))
        env['MAUDE_LIB'] += ':' + os.getcwd()
        if self.debug: 
            print(f"MAUDE_LIB={env['MAUDE_LIB']}")
        # the expawned process:
        self.sh = pexpect.spawn('maude', encoding='utf-8', env=env)
        self.sh.expect('Maude> ')
        self._sync()    

    def _sync(self):
        self.sh.sendline(self._control_request)
        self.sh.expect(self._control_response)
        if self.debug: print(self.sh.before+'->'+self.sh.after)    
            
    def __del__(self):
        if self.debug: print('Destroying Object')
        self.__call__('quit .')
             
    def __call__(self,command,timeout=timeout):
        # Don't terminate session by command
        if command == 'quit .':
            return ''
        # Send command    
        self.sh.sendline(command)
        self._sync()
        #if self.debug: 
        #    print('command: -------------------------------\n'+command)
        #i = self.sh.expect([pexpect.EOF,pexpect.TIMEOUT,self._control_response],timeout)
        #if i == 0:
        #    response = ""
        #elif i == 1:
        #    if self.debug: print('Timeout')
        #    raise(TimeoutException('Timeout'))    
        #else:
            # Note that maude shell makes echo of the sent command. 
            # Remove it from response.
            # response = self.sh.before[len(command)+2:-2]
        response = self.sh.before
        return response
        

Creating maude interpreter:

In [229]:
maude=MaudeInterpreter(debug=False)
type(maude)

__main__.MaudeInterpreter

Load a maude module. Show it and make some reduction:

In [230]:
print(maude('load SIMPLE-NAT .'))
print(maude('show module .'))
print(maude('red s s zero .'))

load SIMPLE-NAT .
Maude> 
show module .
fmod SIMPLE-NAT is
  sort Nat .
  op zero : -> Nat .
  op s_ : Nat -> Nat .
  op _+_ : Nat Nat -> Nat .
  vars M N : Nat .
  eq zero + N = N .
  eq s N + M = s (N + M) .
endfm
Maude> 
red s s zero .
reduce in SIMPLE-NAT : s s zero .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Nat: s s zero
Maude> 


Skip `quit` command:

In [231]:
maude('quit .')

''

Closing maude session on object destroy:

In [232]:
del maude 

## The maude Magic Class

In [233]:
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

In [234]:
# This code can be put in any Python module, it does not require IPython
# itself to be running already.  It only creates the magics subclass but
# doesn't instantiate it yet.
# from __future__ import print_function
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

# The class MUST call this class decorator at creation time
@magics_class
class MaudeMagics(Magics):
    """Adapts Maude Shell to a IPython Magic class.Uses a owned Maude Shell.
       Cell magics are used to execute maude commands.
       Line magics are used for command line options."""  
    def __init__(self,shell):
        # Create the owned Maude Shell instance an pass it as shell
        super(MaudeMagics,self).__init__(MaudeInterpreter(debug=False)) 
        # print("On Construntor:"+str(type(self.shell)))

    @line_cell_magic
    def maude(self, line, cell=None):
        if cell is None:
            return self.shell(line)
        else:
            # We need to split cell in commands and send it one by one 
            # to remove each command echo correctly
            # print(cell)
            # remove trailing newlines 
            print(self.shell(cell))
            
            #for command in cell.split('\n'):#print(self.shell(cell))
            #    # empty commands cause timeout exception
            #    if command: print(self.shell(command)+'\n')


# In order to actually use these magics, you must register them with a
# running IPython.

def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    # You can register the class itself without instantiating it.  IPython will
    # call the default constructor on it.
    ipython.register_magics(MaudeMagics)

Manually executing 'load_ipython_extension' for test purposes:

In [235]:
load_ipython_extension(get_ipython())

Now, MaudeMagic uses an owned MaudeInterpreter to run maude commands:

In [236]:
%%maude
load SIMPLE-NAT .
show module .  
red s s zero .    
    
red s s s zero .

load SIMPLE-NAT .
Maude> show module .  
fmod SIMPLE-NAT is
  sort Nat .
  op zero : -> Nat .
  op s_ : Nat -> Nat .
  op _+_ : Nat Nat -> Nat .
  vars M N : Nat .
  eq zero + N = N .
  eq s N + M = s (N + M) .
endfm
Maude> red s s zero .    
reduce in SIMPLE-NAT : s s zero .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Nat: s s zero
Maude> 
> red s s s zero .
reduce in SIMPLE-NAT : s s s zero .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Nat: s s s zero
Maude> 
> 


## Use cases as test

### JSON Definition

[JSON-LD 1.1](https://www.w3.org/TR/json-ld11/)

In [262]:
%%maude

fmod JSON-BASE is 
    sort JSON .
endfm

view JSON from TRIV to JSON-BASE is 
    sort Elt to JSON .
endv        

fmod JSON is
    --- Section 3.4 Defines JSON Objects 
    protecting STRING .
    protecting RAT . 
    extending JSON-BASE .
    protecting LIST{JSON} * ( sort List{JSON} to List*, sort NeList{JSON} to List+ ) .
    protecting SET{JSON} * ( sort Set{JSON} to Set*, sort NeSet{JSON} to Set+ ) .    
        
    sorts term IRI keyword .

    --- For JSON syntax we use Number, String, Boolean, null, Array and JSON    
    --- subsorts String Rat Bool < JSON . postpose
    subsort String < JSON .
    
    sorts  Pair Pair+ Pair* .
    subsorts Pair < Pair+ < Pair* .
    sort Map .
    subsort Map < JSON .
        
    --- Map 
    op _:_ : String JSON -> Pair [ctor prec 30] .
    op nil : -> Pair* [ctor] .
    op _,_ : Pair* Pair*   -> Pair* [ctor assoc comm id: nil] .
    op _,_ : Pair* Pair+   -> Pair+ [ditto] .

    op {_} : Pair*         -> Map .
         
    --- Core Functionality 
        
    subsorts keyword < String .
    mb "@context"  : keyword .
    mb "@id"       : keyword .
    mb "@type"     : keyword .

endfm


> fmod JSON-BASE is 
>     sort JSON .
> endfm
Advisory: redefining module JSON-BASE.
Maude> 
> view JSON from TRIV to JSON-BASE is 
>     sort Elt to JSON .
> endv        
Advisory: redefining view JSON.
Maude> 
> fmod JSON is
>     --- Section 3.4 Defines JSON Objects 
>     protecting STRING .
>     protecting RAT . 
>     extending JSON-BASE .
>     protecting LIST{JSON} * ( sort List{JSON} to List*, sort NeList{JSON} to List+ ) .
>     protecting SET{JSON} * ( sort Set{JSON} to Set*, sort NeSet{JSON} to Set+ ) .    
> 
>     sorts term IRI keyword .
> 
>     --- For JSON syntax we use Number, String, Boolean, null, Array and JSON    
>     --- subsorts String Rat Bool < JSON . postpose
>     subsort String < JSON .
> 
>     sorts  Pair Pair+ Pair* .
>     subsorts Pair < Pair+ < Pair* .
>     sort Map .
>     subsort Map < JSON .
> 
>     --- Map 
>     op _:_ : String JSON -> Pair [ctor prec 30] .
>     op nil : -> Pair* [ctor] .
>     op _,_ : Pair* Pair*   -> Pair* [ctor assoc

In [263]:
%%maude
red "@type" .

red "@type" .
reduce in JSON : "@type" .
rewrites: 1 in 0ms cpu (0ms real) (~ rewrites/second)
result keyword: "@type"
Maude> 
> 


In [238]:
%%maude
red "Juan" : "Perico" .

red "Juan" : "Perico" .
reduce in JSON : "Juan" : "Perico" .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Pair: "Juan" : "Perico"
Maude> 
> 


In [225]:
%%maude

red 
{
  "name" : "Manu Sporny" ,
  "homepage" : "http://manu.sporny.org/" ,
  "image" : "http://manu.sporny.org/images/manu.png"
} .

In [239]:
%%maude 
--- Example 4
red 
{
  "@context" : {
    "name" : "http://schema.org/name",
    "image" : {
      "@id" : "http://schema.org/image",
      "@type" : "@id"
    },
    "homepage" : {
      "@id" : "http://schema.org/url",
      "@type" : "@id"
    }
  }
} .

--- Example 4
> red 
> {
>   "@context" : {
>     "name" : "http://schema.org/name",
>     "image" : {
>       "@id" : "http://schema.org/image",
>       "@type" : "@id"
>     },
>     "homepage" : {
>       "@id" : "http://schema.org/url",
>       "@type" : "@id"
>     }
>   }
> } .
reduce in JSON : {"@context" : {"name" : "http://schema.org/name", "homepage" :
    {"@id" : "http://schema.org/url", "@type" : "@id"}, "image" : {"@id" :
    "http://schema.org/image", "@type" : "@id"}}} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Map: {"@context" : {"homepage" : {"@id" : "http://schema.org/url",
    "@type" : "@id"}, "image" : {"@id" : "http://schema.org/image", "@type" :
    "@id"}, "name" : "http://schema.org/name"}}
Maude> 
> 


### HTML

## TO-DO
* Remove trailibg newlines grom cell.
* A command that ads a path to MAUDE_LIB

In [25]:
#| hide
import nbdev; nbdev.nbdev_export()